# Text questions

# INTEGRACIÓN BACKEND AI

In [ ]:
!pip install pickle5

     |████████████████████████████████| 132 kB 4.1 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219318 sha256=7e996cc04bf60bc4bde9d6837abb3b269bfd8d3d116fd7dc542ba9c1e7cddfd5
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [ ]:
import random
import os
import pickle5 as pickle
import requests
import pandas as pd
import numpy as np
basic_spotify_url = "https://open.spotify.com/playlist/"

# LINEAS IMPORTANTES DE FIREBASE

# __location__ = os.path.realpath(os.path.join(
#     os.getcwd(), os.path.dirname(__file__))) + '/models/'


def test_model(request):

    question_set = request["set"]
    file_name = "rf_" + "".join([str(i) for i in question_set]) + ".sav"
    answers = request["arrayOfAnswers"]

    # try:
        # LINEAS IMPORTANTES DE FIREBASE
  
        # with open(os.path.join(__location__, file_name), 'rb') as f:
        #     rf_predictor = pickle.loads(f.read())
        #     test_result = rf_predictor.predict([answers])
        #     return str(generate_playlist(test_result, request))
        # return str(generate_playlist(3.26665, request))
    return str(generate_playlist(3.26665, request))

    # except Exception as e:
    #     return "{state:true, err:{msg:{" + str(e) + "}}}"

def generate_playlist(initial_test_result, request):

    token = "Bearer " + request["access_token"]
    
    top_tracks = requests.get("https://api.spotify.com/v1/me/top/tracks", headers={"Authorization": token}).json()
    top_artists = requests.get("https://api.spotify.com/v1/me/top/artists", headers={"Authorization": token}).json()
    songs_ids = []

    for song in top_tracks["items"]:
      album_id = song["album"]["id"]
      url_album = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
      album_songs = requests.get(url_album, headers={"Authorization": token}).json()
      for song in album_songs["items"]:
          songs_ids.append(song["id"])
    
    top_artists_songs_features = np.array([])

    for artist in top_artists["items"]:
        url = f"https://api.spotify.com/v1/artists/{artist['id']}/top-tracks?market=US"
        top_tracks = requests.get(url, headers={"Authorization": token}).json()["tracks"]
        
    for song in top_tracks:
        songs_ids.append(song["id"])

    songs_ids = list(set(songs_ids))
    songs_features = np.array([])

    for i in range(0, len(songs_ids), 100):
        end = i + 100 if len(songs_ids) - i >= 100 else len(songs_ids) 
        batch = songs_ids[i:end]
        ids = ",".join(batch)
        URL = f"https://api.spotify.com/v1/audio-features?ids={ids}"
        batch_songs_features = np.array(requests.get(URL, headers={"Authorization": token}).json()["audio_features"])
        songs_features = np.concatenate((songs_features, batch_songs_features))

    df_songs_complete = pd.DataFrame(songs_features.tolist())
    df_songs = df_songs_complete.drop(["analysis_url", "id", "track_href", "track_href", "uri", "type"], axis=1)
    
    print(df_songs.values.tolist())
    
    # songs_mean = AGARRAR DE LA DATABASE
    # songs_std = AGARRAR DE LA DATABASE
    # normalized_df_songs = (df_songs - songs_mean) / songs_std
    normalized_df_songs = (df_songs - df_songs.mean()) / df_songs.std()

    print("size = ", normalized_df_songs.size)
    print(normalized_df_songs.iloc(0))

    correlative_weights = np.random.rand(13)
    # correlative_weights = AGARRAR DE LA DATABASE
    normalized_songs = normalized_df_songs.values
    scores = normalized_songs.dot(correlative_weights)
    normalized_df_songs["score"] = scores

    number_of_songs = 3
    normalized_df_songs = normalized_df_songs.sort_values("score", ascending=False)
    songs_id = df_songs_complete.loc[normalized_df_songs.index[range(number_of_songs)]]["id"].values
    for song_id in songs_id:
      url_song = f"https://api.spotify.com/v1/tracks/{song_id}"
      song = requests.get(url_song, headers={"Authorization": token}).json()
      print(song["name"] + ", " + song["artists"][0]["name"])

# SI TIRA UNA EXCEPTION CON ALGO DE ["items"] o ["audio_features"] hay que refrescar el access token de spotify que seguró se venció
# aca podes agarrar uno nuevo https://developer.spotify.com/console/get-current-user-top-artists-and-tracks/?type=artists&time_range=&limit=&offset= 
req = {
    "arrayOfAnswers":[0,2,1,2,1,2],
    "set": [
        3,
        6,
        10,
        12,
        16,
    ],
    "access_token":"BQDB76aUXI_ORuUhLjVS4MuwlyRUQJchIFelCplw9pyDEZvI9WToJ11ng5-rIQ2WhfEs3d9fCzT0Ag9O2hzGyLgjbRfSHYONj6mCALOZ2gfQwExsxZCcSFzp_dWH-Far60UFZgRx89xoNjwtBbv8EysuMTSKteHMFOe4kjR7fjhwlsOpK2PHpIfdYuYVeTpGkWicAf5IAUsNKSefb-33lTMiwlMTdTvhmMcEIr75oy8wRMlh0ck"
}

test_model(req)

[[0.743, 0.352, 9.0, -17.225, 0.0, 0.061, 0.146, 0.104, 0.0585, 0.722, 108.785, 350600.0, 4.0], [0.699, 0.61, 2.0, -9.308, 0.0, 0.0541, 0.511, 2.31e-06, 0.127, 0.846, 178.112, 237107.0, 4.0], [0.688, 0.491, 5.0, -7.494, 1.0, 0.269, 0.334, 0.0, 0.0914, 0.737, 79.009, 249771.0, 4.0], [0.537, 0.0498, 10.0, -16.722, 1.0, 0.0239, 0.769, 0.272, 0.105, 0.0692, 59.991, 190379.0, 4.0], [0.892, 0.685, 10.0, -4.906, 0.0, 0.072, 0.301, 0.000744, 0.0775, 0.61, 95.958, 263090.0, 4.0], [0.596, 0.422, 7.0, -9.457, 1.0, 0.0307, 0.725, 1.5e-06, 0.111, 0.642, 179.965, 232333.0, 3.0], [0.595, 0.751, 0.0, -4.711, 0.0, 0.0601, 0.133, 0.0, 0.317, 0.837, 109.493, 175747.0, 4.0], [0.719, 0.222, 10.0, -18.644, 1.0, 0.052, 0.623, 0.91, 0.0991, 0.499, 92.409, 326773.0, 4.0], [0.587, 0.362, 7.0, -7.636, 1.0, 0.0276, 0.199, 1.09e-05, 0.0819, 0.248, 145.961, 340920.0, 4.0], [0.672, 0.875, 1.0, -5.398, 0.0, 0.04, 0.0126, 3.38e-05, 0.135, 0.856, 114.924, 182877.0, 4.0], [0.549, 0.425, 9.0, -15.87, 0.0, 0.0292, 0.404, 

'None'

In [ ]:
arr = np.array()
arr = np.append(arr, 1)
arr

TypeError: ignored

In [ ]:
# después de que haya respondido el segundo test
from scipy.stats import pearsonr

# complete_features_and_scores = np bidimensional array, canciones escuchadas anteriormente + el cambio en el score. Tiene shape (14 x # de canciones escuchadas)
def train_model(initial_test_result, final_test_result, listened_songs_features, complete_features_and_scores):
  result_change = final_test_result - initial_test_result

  listened_songs_features_and_scores = np.concatenate((listened_songs_features, np.ones((listened_songs_features.shape[0], 1))*result_change), axis=1)
  complete_features_and_scores = np.append(complete_features_and_scores, listened_songs_features_and_scores, axis=0)

  scores = complete_features_and_scores[:,13]
  weights = np.array([pearsonr(complete_features_and_scores[:,feature_index], scores)[0] for feature_index in range(13)])

  print(weights)

  # ACTUALIZAR WEIGHTS
  # AGREGAR listened_songs_features_and_scores A complete_features_and_scores EN LA DATABASE

In [ ]:
initial_test_result = 5
final_test_result = 2
listened_songs_features = np.random.rand(3,13)
complete_features_and_scores = np.random.rand(15,14)
train_model(initial_test_result,final_test_result,listened_songs_features,complete_features_and_scores)

[ 0.25964632  0.09126546  0.13837811 -0.31385181  0.07244685 -0.06957284
 -0.06978204  0.26320502 -0.18061766  0.01849117 -0.13893359  0.17591654
  0.26552684]


In [ ]:
import numpy as np
import math
import os
import csv
import pandas as pd

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import Adam

In [ ]:
create_model([0,1,2,3,4,5])

NameError: ignored

In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

def create_model(indexes):
  df = pd.read_excel('/content/dataset.xlsx')
  df = df.drop(columns=df.columns[:4])

  X = df.drop(df.columns[-1], axis=1)
  y = df[df.columns[-1]]

  X_train_complete, X_test_complete, y_train, y_test = train_test_split(X, y, test_size = 0.10, shuffle = True)

  X_train = X_train_complete[X_train_complete.columns[indexes]]
  X_test = X_test_complete[X_train_complete.columns[indexes]]

  # Random Forest
  rf = RandomForestRegressor(n_estimators = 1000)
  rf.fit(X_train, y_train)

  print(mean_absolute_error(y_test, rf.predict(X_test)))

  pickle.dump(rf, open("rf_" + "".join([str(i) for i in indexes]) + ".sav", "wb"))

In [ ]:
indexes = [0,1,2,3,4,5]

create_model(indexes)

3.4273089701432076


In [ ]:
df = pd.read_excel('/content/dataset.xlsx')

In [ ]:
df = df.drop(columns=df.columns[:4])

In [ ]:
indexes = [i for i in range(6)]

In [ ]:
# ======================================
# DIVIDE BETWEEN X AND Y, TRAIN AND TEST
# ======================================
X = df.drop(df.columns[-1], axis=1)
y = df[df.columns[-1]]

X_train_complete, X_test_complete, y_train, y_test = train_test_split(X, y, test_size = 0.10, shuffle = True)

X_train = X_train_complete[X_train_complete.columns[indexes]]
X_test = X_test_complete[X_train_complete.columns[indexes]]

In [ ]:
model = Sequential()
model.add(Dense(5, activation='relu', kernel_regularizer='l2', input_shape=(X_train.shape[1],)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(5, activation='relu', kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(5, activation='relu', kernel_regularizer='l2'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss="mse", metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=20)

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

3/3 [==============================] - 0s 4ms/step - loss: 36.1265 - mean_absolute_error: 4.3402


[36.12651443481445, 4.340174674987793]

In [ ]:
predictions_lr = np.concatenate((X_test_complete, np.transpose([y_test]), np.around(model.predict(X_test))), axis=1).astype(int)

preds = open("/content/predictions_linear_regression.csv", mode="w+", encoding="utf-8")

preds_writer = csv.writer(preds)

preds_writer.writerow(df.columns.tolist() + ["Resultado original", "Resultado predicho"])

for prediction in predictions_lr:
  preds_writer.writerow(prediction)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
pip install pickle5

     |████████████████████████████████| 132 kB 5.3 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219303 sha256=2eb245f8b0e8b9f98e3006a7259c0d2a54825ecf34472ad00232f28dbc028181
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


In [ ]:
generate_playlist("/content/rf_348101115.sav", [[0,3,1,2,1,0]])

In [ ]:
# Random Forest
rf = RandomForestRegressor(n_estimators = 1000)
rf.fit(X_train, y_train)
predictions_rf = np.concatenate((X_test_complete, np.transpose([y_test]), np.transpose([np.around(rf.predict(X_test))])), axis=1).astype(int)

In [ ]:
mean_absolute_error(y_test, rf.predict(X_test))

3.3101753614590703

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

In [ ]:
preds = open("/content/predictions_random_forest.csv", mode="w+")
preds_writer = csv.writer(preds)

preds_writer.writerow(df.columns.tolist() + ["Resultado original", "Resultado predicho"])

for prediction in predictions_rf:
  preds_writer.writerow(prediction)

### Dataset generator

In [ ]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [ ]:
import numpy as np
import math
import os
import csv
from scipy.stats import truncnorm

In [ ]:
os.mkdir("/content/dataset")
f = open("/content/dataset/fake_dataset.csv", mode="w+")
f_writer = csv.writer(f)

f_writer.writerow(["pregunta_1","pregunta_2","pregunta_3","pregunta_4","pregunta_5","pregunta_6","pregunta_7","pregunta_8","pregunta_9","pregunta_10","pregunta_11","pregunta_12","pregunta_13","pregunta_14","pregunta_15","pregunta_16","pregunta_17","pregunta_18","pregunta_19","pregunta_20","pregunta_21","resultado"])

for mean in range(1,5):
  for i in range(2000):
    x = get_truncated_normal(mean,1,0,4)
    numbers = np.around(x.rvs(21)).astype(int)
    f_writer.writerow(np.append(numbers, np.sum(numbers)))

### Prediction model

#### Linear Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import Adam

In [ ]:
def questions_given_indexes(indexes):
  questions = []
  for index in indexes:
    questions.append("pregunta_" + str(index))
  return questions

In [ ]:
indexes = [1,4,6,7,8,15,19]

In [ ]:
questions_given_indexes(indexes)

['pregunta_1',
 'pregunta_4',
 'pregunta_6',
 'pregunta_7',
 'pregunta_8',
 'pregunta_15',
 'pregunta_19']

In [ ]:
# df = pd.read_csv("/content/dataset/fake_dataset.csv")

# # ======================================
# # DIVIDE BETWEEN X AND Y, TRAIN AND TEST
# # ======================================
# # X = df.drop(['resultado'], axis=1)
# X = df[questions_given_indexes(indexes)]
# y = df['resultado']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle = False)

# X_train = np.asarray(X_train).astype(np.float32)
# y_train = np.asarray(y_train).astype(np.float32)
# X_test = np.asarray(X_test).astype(np.float32)
# y_test = np.asarray(y_test).astype(np.float32)

In [ ]:
df = pd.read_csv("/content/dataset/fake_dataset.csv")

# ======================================
# DIVIDE BETWEEN X AND Y, TRAIN AND TEST
# ======================================
X = df.drop(['resultado'], axis=1)
y = df['resultado']

X_train_complete, X_test_complete, y_train, y_test = train_test_split(X, y, test_size = 0.15, shuffle = True)

X_train = X_train_complete[questions_given_indexes(indexes)]
X_test = X_test_complete[questions_given_indexes(indexes)]

In [ ]:
model = Sequential()
model.add(Dense(100, activation='relu', kernel_regularizer='l2', input_shape=(SAMPLE.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss="mse", metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=20)

NameError: ignored

In [ ]:
# estuve buscando y cambiando algunos valores
# si sirve de algo con las configs de arriba da los siguientes valores:
# loss: 39.0153 - mean_absolute_error: 4.3897
# y con las cofiguraciones que puse aca abajo dan:
# loss: 5.2230 - mean_absolute_error: 4.2538

model = Sequential()
model.add(Dense(100, activation='relu', kernel_regularizer='l2', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(150, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(150, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu', kernel_regularizer='l2'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss="mae", metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=20)

In [ ]:
model.evaluate(X_test, y_test, batch_size=32)

37/37 [==============================] - 1s 2ms/step - loss: 5.1213 - mean_absolute_error: 4.2775


[5.121340274810791, 4.277517795562744]

In [ ]:
predictions_lr = np.concatenate((X_test_complete, np.transpose([y_test]), np.around(model.predict(X_test))), axis=1).astype(int)

preds = open("/content/dataset/predictions_linear_regression.csv", mode="w+")
preds_writer = csv.writer(preds)

preds_writer.writerow(["pregunta_1","pregunta_2","pregunta_3","pregunta_4","pregunta_5","pregunta_6","pregunta_7","pregunta_8","pregunta_9","pregunta_10","pregunta_11","pregunta_12","pregunta_13","pregunta_14","pregunta_15","pregunta_16","pregunta_17","pregunta_18","pregunta_19","pregunta_20","pregunta_21","resultado","prediccion"])

for prediction in predictions_lr:
  preds_writer.writerow(prediction)



Probando guardar el modelo en un pickle o en un h5 para exportar e importar en otro codigo.

In [ ]:
#save model
model.save("/content/models/modelo.h5")


In [ ]:
import pickle
import os

archivo = open('/content/pickles/modelo', 'ab+')
pickle.dump(model, archivo, pickle.HIGHEST_PROTOCOL)

archivo.close()


#### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000)
rf.fit(X_train, y_train)
predictions_rf = np.concatenate((X_test_complete, np.transpose([y_test]), np.transpose([np.around(rf.predict(X_test))])), axis=1).astype(int)

In [ ]:
preds = open("/content/dataset/predictions_random_forest.csv", mode="w+")
preds_writer = csv.writer(preds)

preds_writer.writerow(["pregunta_1","pregunta_2","pregunta_3","pregunta_4","pregunta_5","pregunta_6","pregunta_7","pregunta_8","pregunta_9","pregunta_10","pregunta_11","pregunta_12","pregunta_13","pregunta_14","pregunta_15","pregunta_16","pregunta_17","pregunta_18","pregunta_19","pregunta_20","pregunta_21","resultado","prediccion"])

for prediction in predictions_rf:
  preds_writer.writerow(prediction)

[Bayesian Belief Network](https://towardsdatascience.com/bbn-bayesian-belief-networks-how-to-build-them-effectively-in-python-6b7f93435bba)

# VGG 19

[StackOverflow encoding with VGG](https://stackoverflow.com/questions/63465734/how-to-get-the-encoder-from-a-trained-vgg16-network)

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import requests

In [ ]:
model_VGG16 = VGG16(include_top=False, input_shape=input_shape, weights="imagenet")

In [ ]:
model_VGG16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 640, 360, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 640, 360, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 640, 360, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 320, 180, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 320, 180, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 320, 180, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 160, 90, 128)      0     

In [ ]:
def get_encoder(old_model: Model) -> Model:
  # Get encoder
  encoder_input: Model = Model(inputs=old_model.layers[0].input,
                              outputs=old_model.layers[18].output)

  # Create Global Average Pooling.
  encoder_output = encoder_input.layers[-1].output

  # Create the encoder adding the GAP layer as output.
  encoder: Model = Model(encoder_input.input, encoder_output, name='encoder')

  return encoder

In [ ]:
img_url = "https://d25rq8gxcq0p71.cloudfront.net/dictionary-images/324/small.jpg"
img = Image.open(requests.get(img_url, stream=True).raw)
input_shape_from_url = (img.height, img.width, 3)

encoder = get_encoder(VGG16(include_top=False, input_shape=input_shape_from_url, weights="imagenet"))
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 216, 324, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 216, 324, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 216, 324, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 108, 162, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 108, 162, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 108, 162, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 54, 81, 128)       0   

In [ ]:
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

encoded_x = encoder.predict(x)

In [ ]:
encoded_x

array([[[[ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ]],

        [[ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.

In [ ]:
def generate_encoder(input_shape):
     model = VGG16(include_top=False, input_shape=input_shape, weights="imagenet")
     F1 = Flatten()(model.get_layer(index=1).output)
     F2 = Flatten()(model.get_layer(index=2).output)
     F3 = Flatten()(model.get_layer(index=3).output)
     F4 = Flatten()(model.get_layer(index=4).output)
     F5 = Flatten()(model.get_layer(index=5).output)   
     M = Model(model.inputs,[F1,F2,F3,F4,F5])  
     return M 

In [ ]:
img_url = "https://d25rq8gxcq0p71.cloudfront.net/dictionary-images/324/small.jpg"
img = Image.open(requests.get(img_url, stream=True).raw)
input_shape_from_url = (img.height, img.width, 3)

encoder = generate_encoder(input_shape_from_url)
encoder.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 216, 324, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 216, 324, 64) 1792        input_13[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 216, 324, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 108, 162, 64) 0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [ ]:
mx = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

encoded_x = encoder.predict(x)

In [ ]:
print(f"({len(encoded_x)}, {len(encoded_x[0])}, {len(encoded_x[0][0])})")

NameError: ignored

# Preparar el dataset para la DA 

In [ ]:
import pandas as pd

df = pd.DataFrame(pd.read_excel("/content/dataset/revibe.xlsx"))

In [ ]:
df = df.drop(['Marca temporal', '¿Cuántos años tenés?', '¿De dónde sos? (ciudad y país)', '¿Cuál es tu género?'], axis=1)

In [ ]:
df.to_csv("/content/dataset/dataset.csv", index = None, header=True, encoding="utf-8")

# Data Augmentation with a GAN



In [ ]:
import csv
import numpy
import numpy as np
from numpy.random import uniform
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
BATCH = 16
SAMPLE_LEN = 19       # number N of points where a curve is sampled
SAMPLE_SIZE = 919   # number of curves in the training set

In [ ]:
df_x = pd.read_excel('/content/dataset/dataset.xlsx')

In [ ]:
df_x = df_x.drop(columns=df_x.columns[:4])

In [ ]:
reader = csv.reader(open("/content/dataset/dataset.csv"), delimiter=",")
next(reader)
x = np.array(list(reader)).astype("float")

In [ ]:
positive = [0,1,4,6,8,9,13,14,17,18]
negative = [i for i in range(20) if i not in positive]

In [ ]:
arr_x = np.array(df_x).astype("float")

In [ ]:
arr_x[[positive, ]]

(919, 20)

In [ ]:
arr_x_test = np.sum(arr_x, axis=1)

In [ ]:
o = np.array([1,2,3,4,5])
o[[2,4]]

array([3, 5])

In [ ]:
df_x

,Me siento calmado,Me siento seguro,Estoy tenso,Estoy contrariado (malhumorado o disgustado),Me siento cómodo (estoy a gusto),Estoy preocupado ahora por posibles desgracias futuras,Me siento descansado,Me siento angustiado,Me siento confortable,Tengo confianza en mi mismo,Me siento nervioso,Estoy desasosegado (agitado e inquieto),Me siento muy “atado” (como oprimido),Estoy relajado,Me siento satisfecho,Estoy preocupado,Me siento aturdido y sobreexcitado,Me siento alegre,En este momento me siento bien,Me siento alterado
0,0,1,3,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1
1,2,2,0,0,2,0,1,1,2,3,1,0,0,2,2,1,0,2,2,0
2,2,3,0,1,3,1,2,0,2,3,0,0,0,2,2,0,0,3,3,1
3,2,2,1,1,3,1,1,0,2,3,1,0,1,2,2,1,0,2,3,1
4,2,3,0,1,3,2,3,0,2,3,1,0,2,3,2,1,0,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,1,1,3,1,1,3,1,2,1,1,3,3,1,0,1,3,2,1,1,2
915,1,2,1,1,2,0,1,1,1,1,0,1,1,2,2,0,1,1,2,1
916,2,3,0,0,2,0,0,0,2,3,1,0,0,2,2,0,0,2,3,0
917,2,1,1,1,1,0,1,0,1,2,1,1,0,2,2,1,0,2,2,1


In [ ]:
for raw in x:
  for repeat in range(10):
    new_raw = raw
    index_alterate = random.randint(0,18)
    if (raw[index_alterate] == 0): new_raw[index_alterate] += 1
    elif (raw[index_alterate] == 3): new_raw[index_alterate] -= 1
    else:
      change = random.randrange(-1,2,2)
      print(type(new_raw[index_alterate]))
      new_raw[index_alterate] += change
    x.append(new_raw)

AttributeError: ignored

In [ ]:
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, LeakyReLU
from keras.models import Sequential
import numpy as np
from numpy.random import uniform
import random

LEAKY_RELU = LeakyReLU(0.3)
DROPOUT = Dropout(0.4)
ADAM = Adam(learning_rate=0.001, beta_1=.5) #previously 1

In [ ]:
generator = Sequential()
generator.add(Dense(SAMPLE_LEN))
generator.add(LEAKY_RELU)
generator.add(DROPOUT) #a
generator.add(Dense(SAMPLE_LEN))
generator.add(LEAKY_RELU)
generator.add(Dense(SAMPLE_LEN))
generator.compile(optimizer = ADAM, loss = "mse", metrics = ["accuracy"])

In [ ]:
discriminator = Sequential()
discriminator.add(Dense(SAMPLE_LEN, activation="relu"))
discriminator.add(DROPOUT)
discriminator.add(Dense(SAMPLE_LEN, activation="relu"))
discriminator.add(DROPOUT)
discriminator.add(Dense(1, activation = "sigmoid"))
discriminator.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
EPOCHS = 64
 
NOISE = (np.random.rand(SAMPLE_SIZE, 19) * 4).astype(int)
ONES = np.ones((SAMPLE_SIZE))
ZEROS = np.zeros((SAMPLE_SIZE))
print("epoch | dis. loss | dis. acc | gen. loss | gen. acc")
print("------+-----------+----------+-----------+----------")

for e in range(EPOCHS):
    for k in range(SAMPLE_SIZE//BATCH):
        n = np.random.randint(0, SAMPLE_SIZE, size = BATCH)
        p = generator.predict(NOISE[n])
        x = np.concatenate((SAMPLE[n], p))
        y = np.concatenate((ONES[n], ZEROS[n]))
        d_result = discriminator.train_on_batch(x, y)
        discriminator.trainable = False
        g_result = gan.train_on_batch(NOISE[n], ONES[n])
        discriminator.trainable = True
    print(f" {e:04n} |  {d_result[0]:.5f}  |  {d_result[1]:.5f} |  {g_result[0]:.5f}  |  {g_result[1]:.5f}")

In [ ]:
print(generator.predict(np.array([NOISE[0]])).astype(int)[0])
print(generator.predict(np.array([NOISE[15]])).astype(int)[0])
print(generator.predict(np.array([NOISE[29]])).astype(int)[0])
print(generator.predict(np.array([NOISE[34]])).astype(int)[0])
print(generator.predict(np.array([NOISE[56]])).astype(int)[0])
print(generator.predict(np.array([NOISE[78]])).astype(int)[0])
print(generator.predict(np.array([NOISE[12]])).astype(int)[0])
print(generator.predict(np.array([NOISE[9]])).astype(int)[0])

[ -69  614 -123 -141   25 -326  205  372  152  -39  167   51  -26  528
  131   84  238   31 -238]
[ -79  699 -142 -161   26 -367  232  426  173  -43  187   60  -27  603
  149   91  269   31 -269]
[ -75  682 -138 -156   26 -357  227  414  170  -43  181   58  -27  589
  146   89  262   30 -262]
[ -54  488 -100 -112   18 -256  162  297  121  -30  129   42  -20  422
  104   63  187   20 -188]
[ -52  475  -96 -109   18 -251  158  287  117  -30  127   38  -20  410
  102   64  184   22 -183]
[ -50  447  -92 -102   17 -236  149  272  111  -28  120   39  -19  387
   94   59  171   20 -173]
[ -67  606 -123 -140   22 -318  201  368  150  -37  161   51  -23  523
  130   79  234   25 -233]
[ -49  439  -89 -101   17 -231  145  268  109  -27  117   38  -17  380
   94   56  169   19 -169]


In [ ]:
y

array([-297,  314,  -97,  787, -590,  -50,  545, -379, -393,  942,   69,
        155,   50, -354,  129, -388,  333,  338, -367])

## usanndo librería river.py

[Docs](https://riverml.xyz/latest/)

In [ ]:
pip install river

     |████████████████████████████████| 2.4 MB 4.1 MB/s 


In [ ]:
from river import datasets

dataset = datasets.Bikes()
dataset

ImportError: ignored

# Experimenting with autoencoders

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D

In [ ]:
shape_img = (256,256,3)

n_hidden_1, n_hidden_2, n_hidden_3 = 16, 8, 8
convkernel = (3, 3)  # convolution kernel
poolkernel = (2, 2)  # pooling kernel

In [ ]:
input = Input(shape=shape_img)
x = Conv2D(n_hidden_1, convkernel, activation='relu', padding='same')(input)
x = MaxPooling2D(poolkernel, padding='same')(x)
x = Conv2D(n_hidden_2, convkernel, activation='relu', padding='same')(x)
x = MaxPooling2D(poolkernel, padding='same')(x)
x = Conv2D(n_hidden_3, convkernel, activation='relu', padding='same')(x)
encoded = MaxPooling2D(poolkernel, padding='same')(x)

x = Conv2D(n_hidden_3, convkernel, activation='relu', padding='same')(encoded)
x = UpSampling2D(poolkernel)(x)
x = Conv2D(n_hidden_2, convkernel, activation='relu', padding='same')(x)
x = UpSampling2D(poolkernel)(x)
x = Conv2D(n_hidden_1, convkernel, activation='relu')(x)
x = UpSampling2D(poolkernel)(x)
decoded = Conv2D(shape_img[2], convkernel, activation='sigmoid', padding='same')(x)

autoencoder = tf.keras.Model(input, decoded)

In [ ]:
autoencoder.compile(optimizer="adam", loss="binay_crossentropy")

In [ ]:
n_epochs = 50
batch_size = 256

In [ ]:
indices_fracs = split(fracs=[0.9, 0.1], N=len(X), seed=0)
X_train, X_valid = X[indices_fracs[0]], X[indices_fracs[1]]
self.autoencoder.fit(X_train, X_train, epochs = n_epochs, batch_size = batch_size, shuffle = True, validation_data = (X_valid, X_valid))

KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pickle
import random
import os
import pickle5 as pickle
import requests
import numpy as np
basic_spotify_url = "https://open.spotify.com/playlist/"

playlist_ids = {
    "triste_0": "4uEUHMAezybrAOKy7kfKYR",
    "triste_1": "2SN6JTq4YbPigA7ZsHSuAD",
    "triste_2": "7u3DeMpkH4seXT8MBvGtIT",
    "estable_0": "3Qpwqo89iRx1P8QZ2ClgH4",
    "estable_1": "5wQaXaytwCof9fUGbqRwyL",
    "estable_2": "63M22uFRIkmofA3N1zPdpq",
    "alegre_0": "0xHgSdTJkzDP6vX3pKIbLF",
    "alegre_1": "0t9UUOQeGYWxhCqSpUnxGN",
    "alegre_2": "63M22uFRIkmofA3N1zPdpq"
}

# __location__ = os.path.realpath(os.path.join(
#     os.getcwd(), os.path.dirname(__file__))) + '/models/'


def test_model(request):

    question_set = request["set"]
    file_name = "rf_" + "".join([str(i) for i in question_set]) + ".sav"
    answers = request["arrayOfAnswers"]

    try:
        # with open(os.path.join(__location__, file_name), 'rb') as f:
        #     rf_predictor = pickle.loads(f.read())
        #     test_result = rf_predictor.predict([answers])

        #     #---
        #     return str(generate_playlist(test_result, request))
        return str(generate_playlist(3.26665, request))

    except Exception as e:
        return str({"exception": str(e)})


def generate_playlist(test_result, request):

    token = "Bearer " + request["access_token"]
    
    top_tracks = requests.get("https://api.spotify.com/v1/me/top/tracks", headers={"Authorization": token}).json()
    top_artists = requests.get("https://api.spotify.com/v1/me/top/artists", headers={"Authorization": token}).json()
    songs_features = np.array([])

    for song in top_tracks["items"]:
      
      album_id = song["album"]["id"]
      url_album = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
      album_songs = requests.get(url_album, headers={"Authorization": token}).json()
      ids = ""
      
      for song in album_songs["items"]:
          ids += song["id"] + ","
      ids = ids[:-1]
      url_new_songs_features = f"https://api.spotify.com/v1/audio-features?ids={ids}"
      new_songs_features = np.array(requests.get(url_new_songs_features, headers={"Authorization": token}).json()["audio_features"])
      songs_features = np.concatenate((songs_features,new_songs_features))
    
    top_artists_songs_features = np.array([])

    for artist in top_artists["items"]:
        url = f"https://api.spotify.com/v1/artists/{artist['id']}/top-tracks?market=US"
        top_tracks = requests.get(url, headers={"Authorization": token}).json()["tracks"]
        # print(top_tracks)
        #hacer el for devuelta
        
        for song in top_tracks:
            ids += song["id"] + ","
        ids = ids[:-1]
        url_new_songs_features = f"https://api.spotify.com/v1/audio-features?ids={ids}"
        new_songs_features = np.array(requests.get(url_new_songs_features, headers={"Authorization": token}).json()["audio_features"])
        top_artists_songs_features = np.concatenate((songs_features,new_songs_features))

    print(top_artists_songs_features)

    random_playlist = random.randint(0, 2)
    test_result = float(test_result)

    if (test_result < 0):
        # triste
        return {"test_result": test_result, "playlist_link": basic_spotify_url + playlist_ids["triste_" + str(random_playlist)]}
    elif (test_result < 10):
        # mediano
        return {"test_result": test_result, "playlist_link":  basic_spotify_url + playlist_ids["estable_" + str(random_playlist)]}
    else:
        # contento
        return {"test_result": test_result, "playlist_link":  basic_spotify_url + playlist_ids["alegre_" + str(random_playlist)]}

req = {
    "arrayOfAnswers":[0,2,1,2,1,2],
    "set": [
        3,
        6,
        10,
        12,
        16,
    ],
    "access_token":"BQDvR6b6hS4JHqLUQpS7B57loBVKSNwxp-9CR5WRLsI0gNUpO4Zri9768FgJiXXs8t3eTSgg7F7E5YHCq_GTh6hznuQHOWKxGgVF9AJiwR8MluuBHTrcaOp92NAVQ16CuLz2HQThMYWvMGybVrrAlih3urtd9JMMYQU4iy9S_VOGMELZfQeErG2So1Zz89VkgMeoAqwtllGdKsoCPkW5I0xwb6Y"
}


print(test_model(req))


{'audio_features': [{'danceability': 0.648, 'energy': 0.792, 'key': 0, 'loudness': -6.058, 'mode': 1, 'speechiness': 0.035, 'acousticness': 0.0657, 'instrumentalness': 5.31e-06, 'liveness': 0.0788, 'valence': 0.935, 'tempo': 159.968, 'type': 'audio_features', 'id': '6Vk5QUhQ0v5OjYQIYvL5VL', 'uri': 'spotify:track:6Vk5QUhQ0v5OjYQIYvL5VL', 'track_href': 'https://api.spotify.com/v1/tracks/6Vk5QUhQ0v5OjYQIYvL5VL', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6Vk5QUhQ0v5OjYQIYvL5VL', 'duration_ms': 260867, 'time_signature': 4}, {'danceability': 0.573, 'energy': 0.658, 'key': 1, 'loudness': -5.557, 'mode': 0, 'speechiness': 0.0396, 'acousticness': 0.0195, 'instrumentalness': 0, 'liveness': 0.209, 'valence': 0.481, 'tempo': 105.052, 'type': 'audio_features', 'id': '0uCKawvDKjr0AegelrzSuX', 'uri': 'spotify:track:0uCKawvDKjr0AegelrzSuX', 'track_href': 'https://api.spotify.com/v1/tracks/0uCKawvDKjr0AegelrzSuX', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0uCKawvDKjr0A